In [1]:
import typesense
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']

In [3]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

In [13]:
search_parameters = {
  'q'         : 'ghibli studio',
   'query_by'  : 'name',
#   'filter_by' : 'num_employees:>100',
#   'sort_by'   : 'num_employees:desc'
}

search_hits = client.collections['entity'].documents.search(search_parameters)

In [14]:
search_hits

{'facet_counts': [],
 'found': 1,
 'hits': [{'document': {'id': 'studio/21',
    'name': '"Studio Ghibli"',
    'type': ['Studio']},
   'highlight': {'name': {'matched_tokens': ['Studio', 'Ghibli'],
     'snippet': '"<mark>Studio</mark> <mark>Ghibli</mark>"'}},
   'highlights': [{'field': 'name',
     'matched_tokens': ['Studio', 'Ghibli'],
     'snippet': '"<mark>Studio</mark> <mark>Ghibli</mark>"'}],
   'text_match': 1157451471441625209,
   'text_match_info': {'best_field_score': '2211897868544',
    'best_field_weight': 15,
    'fields_matched': 1,
    'score': '1157451471441625209',
    'tokens_matched': 2}}],
 'out_of': 9637,
 'page': 1,
 'request_params': {'collection_name': 'entity',
  'per_page': 10,
  'q': 'ghibli studio'},
 'search_cutoff': False,
 'search_time_ms': 0}